## import libraries

In [12]:
from compas.geometry import Frame, Plane
from compas.geometry import Box, Sphere
from compas_vol.primitives import VolBox, VolSphere
from compas_vol.combinations import Union, SmoothUnion, Intersection
from compas_vol.modifications import Shell

In [13]:
import numpy as np
import meshplot as mp
from skimage.measure import marching_cubes
from compas_vol.utilities import get_random_vector_3D, bbox_edges

## create volumetric object (CSG tree)

In [22]:
#generate random points around a sphere
nbrPts = 20
sphereRad = 3.4
pts = [get_random_vector_3D(sphereRad) for _ in range(nbrPts)]
print(pts)

[(2.766175296576979, -1.0671202033830507, -1.6641901033652908), (3.2112307205824195, -0.32130357576106355, -1.0699351715832446), (-0.0799724467819307, -3.399016390030881, -0.01708765803586938), (0.7683923455070174, 0.5885803390540784, -3.259316859074185), (0.6048189322132206, -1.6148820410929599, -2.9302474387996478), (-2.005523871311858, -2.416877580726789, 1.3025270674264393), (-1.2137529019492768, 3.1571325880586483, 0.34542396330858455), (2.2750161154458, 0.2321241932834713, 2.5160325978322295), (-2.1696811151286535, -1.3231219651509285, -2.2587235607725944), (2.087081042446428, -1.6297984169783526, 2.132568836468259), (0.8053205146382563, 2.1780089765158297, -2.4834926548953677), (-2.5279634227741834, -0.5365555722993888, -2.2094137346704237), (0.9756060411708112, 3.244199073901206, -0.28873036094178467), (-2.8416469468339702, -1.8658472613462498, -0.06047004941187737), (-1.5875044722425207, -2.4232380250782444, 1.7798165704433946), (-1.4910216948403847, -2.4451677622579857, 1.832

In [23]:
#create volumetric boxes and unify them
volBoxes = []
for [x,y,z] in pts:
    plane = Plane([x,y,z],[x,y,z])
    box = Box(Frame.from_plane(plane),1.2,1.2,3.5)
    vbox = VolBox(box, 0.1)
    volBoxes.append(vbox)
uvb = Union(volBoxes)

In [24]:
#combine core and branches
vs = VolSphere(Sphere([0,0,0], 2.0))
su = SmoothUnion(uvb, vs, 2.0)

In [25]:
#thicken shell and cull extremities
s = Shell(su, 0.2, 1.0)
node = Intersection(VolSphere(Sphere([0,0,0], 4.8)), s)

## workspace (dense grid)

In [26]:
#workspace initialization
# lower and upper bounds
lbx, ubx = -5.0, 5.0
lby, uby = -5.0, 5.0
lbz, ubz = -5.0, 5.0
# resolution(s)
nx, ny, nz = 100, 100, 100
x, y, z = np.ogrid[lbx:ubx:nx*1j, lby:uby:ny*1j, lbz:ubz:nz*1j]
#voxel dimensions
gx = (ubx-lbx)/nx
gy = (uby-lby)/ny
gz = (ubz-lbz)/nz

## sample at discrete interval

In [27]:
dm = node.get_distance_numpy(x, y, z)

## generate isosurface (marching cube)

In [28]:
v, f, n, l = marching_cubes(dm, 0, spacing=(gx, gy, gz))
v += [lbx,lby,lbz]

## display mesh

In [29]:
p = mp.plot(v, f, c=np.array([0,0.57,0.82]), shading={"flat":False, "roughness":0.4, "metalness":0.01, "reflectivity":1.0})
vs,ve = bbox_edges(lbx,ubx,lby,uby,lbz,ubz)
p.add_lines(np.array(vs), np.array(ve))
p.add_points(np.array([[lbx,lby,lbz]]))

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(-0.021509…

2